In [1]:
!mkdir -p /tmp/pip/cache/
!cp /kaggle/input/wpca-01-py3-whl/wpca-0.1-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ wpca

Looking in links: /tmp/pip/cache/
Processing /tmp/pip/cache/wpca-0.1-py3-none-any.whl


In [2]:
import os
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from wpca import WPCA
from sklearn.decomposition import PCA
import gc
import polars

In [3]:
def normalize_targets(y):
    y_min = np.array([0.0, 0.0], dtype=np.float32)  # azimuth: [0, 2*pi], zenith: [0, pi]
    y_max = np.array([2 * np.pi, np.pi], dtype=np.float32)
    return (y - y_min) / (y_max - y_min)

def denormalize_targets(y_norm):
    y_min = np.array([0.0, 0.0], dtype=np.float32)
    y_max = np.array([2 * np.pi, np.pi], dtype=np.float32)
    return y_norm * (y_max - y_min) + y_min

def normalize_features(df, features):
    df = df.copy()
    cols_min = df[features].min()
    cols_max = df[features].max()
    df[features] = (df[features] - cols_min) / (cols_max - cols_min)
    return df

In [4]:
# TO TEST DATASET
data_dir = '/kaggle/input/icecube-neutrinos-in-deep-ice/'
metadata_file = f'{data_dir}/train_meta.parquet'
batch_files = [f'{data_dir}train/batch_{i}.parquet' for i in range(1, 2)] # You can set up to 660 files

sensor_geometry = pd.read_csv(os.path.join(data_dir, 'sensor_geometry.csv'))
metadata = pd.read_parquet(metadata_file)

In [5]:
def generating_some_features(batch_file, metadata, sensor_geometry):
    
    
    def join_tables_all(df_meta, df_batch, df_sensor):
        return df_meta.join(df_batch, on='event_id').join(df_sensor, on='sensor_id').with_columns([
            (polars.col('time') - polars.col('time').min()).over('event_id')
        ])

    def generate_features_grouped(dataf):
        return dataf.groupby('event_id').agg([
        polars.col('x').mean().alias('x_mean'),
        polars.col('x').median().alias('x_median'),
        polars.col('y').mean().alias('y_mean'),
        polars.col('y').median().alias('y_median'),
        polars.col('z').mean().alias('z_mean'),
        polars.col('z').median().alias('z_median'),    
        polars.col('time').mean().alias('event_mean_time'),
        polars.col('time').max().alias('event_max_time'),
        polars.col('charge').min().alias('event_min_charge'),
        polars.col('charge').mean().alias('event_mean_charge'),
        polars.col('charge').max().alias('event_max_charge'),
        polars.col('charge').count().alias('overall_count'),
        polars.col('auxiliary').sum().alias('overall_aux_sum'),
        polars.col('charge').sum().alias('sum_charge'),
        (polars.col('auxiliary').sum() / polars.col('auxiliary').count()).alias('aux_ratio'),
        polars.col('sensor_id').n_unique().alias('sensor_count'),
    ])
    
    def add_ranks(dataf):
        return dataf.with_columns(
[
    polars.col('time').rank('ordinal').over('event_id').alias('time_rank_asc'),
    polars.col('time').rank('ordinal', descending=True).over('event_id').alias('time_rank_des'),
    polars.col('charge').rank('ordinal').over('event_id').alias('charge_rank_asc'),
    polars.col('charge').rank('ordinal').over('event_id').alias('charge_rank_des')
])

    def make_geometrical_features(dataf):
        geometrical_features = dataf.select('event_id').unique()
        for direction in ['time_rank_asc','time_rank_des', 'charge_rank_asc', 'charge_rank_des']:
            for direction_axis in ['x', 'y', 'z']:    
                temp_col_1 = dataf.filter(polars.col(direction) == 1).select([
                    polars.col('event_id'),
                    polars.col(direction_axis).over('event_id')
                ]).with_columns([
                    polars.col(direction_axis).alias(direction_axis+'_'+direction+'_1')]
                ).select(polars.col('event_id'), polars.col(direction_axis+'_'+direction+'_1'))

                temp_col_2 = dataf.filter(polars.col("time_rank_asc") == 2).select([
                    polars.col('event_id'),
                    polars.col(direction_axis).over('event_id')
                ]).with_columns([
                    polars.col(direction_axis).alias(direction_axis+'_'+direction+'_2')]
                ).select(polars.col('event_id'), polars.col(direction_axis+'_'+direction+'_2'))

                temp_col_3 = dataf.filter(polars.col("time_rank_asc") == 3).select([
                    polars.col('event_id'),
                    polars.col(direction_axis).over('event_id')
                ]).with_columns([
                    polars.col(direction_axis).alias(direction_axis+'_'+direction+'_3')]
                ).select(polars.col('event_id'), polars.col(direction_axis+'_'+direction+'_3'))

                geometrical_features = geometrical_features.join(temp_col_1, on='event_id', how='left'
                               ).join(temp_col_2, on='event_id', how='left'
                               ).join(temp_col_3, on='event_id', how='left'
                               )
        return geometrical_features.fill_null(1000).fill_nan(1000)

    train_batch = polars.scan_parquet(batch_file).lazy()
    df_train_meta = polars.DataFrame(metadata).lazy()
    df_sensor_geometry = polars.DataFrame(sensor_geometry).with_columns(polars.col('sensor_id').cast(polars.Int16)).lazy()
    
    #Not accounting for aux
    features_grouped_metrics = df_train_meta.pipe(join_tables_all, train_batch, df_sensor_geometry
                      ).pipe(generate_features_grouped).collect()

    geometrical_features = df_train_meta.pipe(join_tables_all, train_batch, df_sensor_geometry
                      ).pipe(add_ranks
                      ).collect().pipe(make_geometrical_features)

    temp_1 = features_grouped_metrics.join(geometrical_features, on='event_id', how='left')


    #AUX = FALSE

    features_grouped_metrics = df_train_meta.pipe(join_tables_all, train_batch, df_sensor_geometry
                      ).filter(polars.col('auxiliary') == False).pipe(generate_features_grouped).collect()

    geometrical_features = df_train_meta.pipe(join_tables_all, train_batch, df_sensor_geometry
                      ).filter(polars.col('auxiliary') == False).pipe(add_ranks
                      ).collect().pipe(make_geometrical_features)

    temp_2 = features_grouped_metrics.join(geometrical_features, on='event_id', how='left')
    
    temp_3 = temp_1.join(temp_2, on = 'event_id', how='left').fill_null(0).fill_nan(0)
    del temp_1, temp_2, features_grouped_metrics, geometrical_features
    
    temp_3 = temp_3.to_pandas().set_index('event_id')
    
    temp_3 = (temp_3-temp_3.mean())/temp_3.std()
    
    return temp_3

In [6]:
class NeutrinoDataset(Dataset):
    def __init__(self, metadata, sensor_geometry, batch_file, mode="train"):
        self.metadata = metadata
        self.sensor_geometry = normalize_features(sensor_geometry, ['x', 'y', 'z'])       
        self.batch_data = pd.read_parquet(batch_file)       
        self.mode = mode
        self.get_time_valid_length()

        self.batch_features = generating_some_features(batch_file, metadata, sensor_geometry)
        
    def __len__(self):
        return len(self.metadata)

    
    def __getitem__(self, idx):
        event = self.metadata.iloc[idx]
        event_data = self.batch_data.loc[event.event_id]
        if len(event_data) > 1010:
            event_data = event_data.sample(1000)
        features = self.combine_features(event_data, idx)
        
        if self.mode == "train":
            y = np.array([event['azimuth'], event['zenith']], dtype=np.float32)
            y_norm = normalize_targets(y)
            return features, y_norm
        else:
            return event.event_id, features
        

    # Return dataframe with PCA and WPCA predicted angles (using all and non-aux pulses) for single event
    def process_event_pca(self, event_pulses):
        event_pulses = event_pulses.reset_index(drop=True)
        event_pulses['auxiliary'] = event_pulses['auxiliary'] * 1
        event_pulses["time_from_start"] = event_pulses.time.values - event_pulses.time.min()
        time_peak_charge = event_pulses.loc[event_pulses['charge'].argmax(), 'time_from_start']
        time_valid_min = time_peak_charge - self.time_valid_length
        time_valid_max = time_peak_charge + self.time_valid_length
        time_valid = ((event_pulses['time_from_start'] > time_valid_min) * 
                      (event_pulses['time_from_start'] < time_valid_max))
        event_pulses['pulse_rank'] = 2 * (1 - event_pulses['auxiliary'].values) + (time_valid)
        event_pulses = event_pulses.drop('time_from_start', axis=1)

        event_pulses[['x', 'y', 'z']] = self.sensor_geometry.loc[event_pulses.sensor_id].reset_index()[['x', 'y', 'z']]
        
        event_pulses.x = event_pulses.x - event_pulses.x.agg('mean')
        event_pulses.y = event_pulses.y - event_pulses.y.agg('mean')
        event_pulses.z = event_pulses.z - event_pulses.z.agg('mean')
        

        result = []
        result.extend(self.get_angles(event_pulses, 'PCA', drop_aux=True))
        result.extend(self.get_angles(event_pulses, 'PCA', drop_aux=False))
        result.extend(self.get_angles(event_pulses, 'WPCA', drop_aux=True))
        result.extend(self.get_angles(event_pulses, 'WPCA', drop_aux=False))
        result.extend(self.get_angles(event_pulses, 'WPCA_new_weights', drop_aux=True))
        result.extend(self.get_angles(event_pulses, 'WPCA_new_weights', drop_aux=False))

        return np.array(result)
    
    
    def get_time_valid_length(self):
        c_const = 0.299792458  # speed of light [m/ns]

        x_min, x_max = self.sensor_geometry.x.agg([min, max]).values
        y_min, y_max = self.sensor_geometry.y.agg([min, max]).values
        z_min, z_max = self.sensor_geometry.z.agg([min, max]).values

        detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
        self.time_valid_length = detector_length / c_const
    
    
    def get_weight_all(self, pulse_rank):
        if pulse_rank == 2:
            return 1
        if pulse_rank == 3:
            return 5
        else:
            return 0.001
    
    
    # Calculate target angles for single event
    def get_angles(self, event_pulses, ThisPCA='WPCA', drop_aux=False):
        num_of_nonaux_pulses = event_pulses.auxiliary.count() - event_pulses.auxiliary.sum()
        
        # Computing the weighted PCA
        if ThisPCA == 'WPCA':
            ThisPCA = WPCA
            #if it is necessary to throw out auxiliary pulses
            if drop_aux and num_of_nonaux_pulses > 5:
                event_pulses = event_pulses.loc[event_pulses.auxiliary == 0].copy()
                weights = np.stack([event_pulses.pulse_rank, event_pulses.pulse_rank, \
                                   event_pulses.pulse_rank], axis=1)
            else:
                weights = np.stack([event_pulses.pulse_rank, event_pulses.pulse_rank, \
                                   event_pulses.pulse_rank], axis=1)
            kwds = {'weights': weights}
            
        elif ThisPCA == 'WPCA_new_weights':
            ThisPCA = WPCA
            event_pulses['weights_all'] = event_pulses['pulse_rank'].apply(self.get_weight_all)
            #if it is necessary to throw out auxiliary pulses
            if drop_aux and num_of_nonaux_pulses > 5:
                event_pulses = event_pulses.loc[event_pulses.auxiliary == 0].copy()
                weights = np.stack([event_pulses.weights_all, event_pulses.weights_all, \
                                   event_pulses.weights_all], axis=1)
            else:
                weights = np.stack([event_pulses.weights_all, event_pulses.weights_all, \
                                   event_pulses.weights_all], axis=1)
            kwds = {'weights': weights}

        # Computing the stardard PCA
        else:
            ThisPCA = PCA
            kwds = {}
            if drop_aux and num_of_nonaux_pulses > 5:
                event_pulses = event_pulses.loc[event_pulses.auxiliary == 0].copy()

        # Computing the PCA vectors
        X = event_pulses.loc[:, ['x', 'y', 'z']]
        pca = ThisPCA().fit(X, **kwds)
            
        if len(pca.components_) > 2:
            v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8, v_9 =  map(float, np.array(pca.components_[:3]).reshape(-1, 1))
        else:
            v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8, v_9 =  np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
                
        # Calculating the first principal component vector
        vector = pca.components_[0]

        # Calculating a distance of a point from a plane
        event_pulses['distance'] = (event_pulses.x * vector[0] +
                                        event_pulses.y * vector[1] +
                                        event_pulses.z * vector[2]) / np.linalg.norm(vector)

        # Flip the vector direction if it points away from the neutrino origin
        if (event_pulses.loc[event_pulses.distance > 0].time.mean() >
                event_pulses.loc[event_pulses.distance < 0].time.mean()):
            vector = -1 * np.array(vector)

        vector = np.clip(vector, -1, 1)

        # Calculating the angles
        zenith = np.arccos(vector[2])
        azimuth = np.arctan2(vector[1], vector[0])
        if azimuth < 0:
            azimuth = 2 * np.pi + azimuth

        azimuth = azimuth / (2 * np.pi)
        zenith = zenith / np.pi

        return float(zenith), float(azimuth), v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8, v_9

    
    def combine_features(self, event_data, idx):
        pca_features = self.process_event_pca(event_data)
        batch_features = self.batch_features.loc[self.metadata.iloc[idx]['event_id']].fillna(0).values

        combined_features = np.hstack([
            pca_features,
            batch_features
        ])
        assert np.all(~np.isnan(combined_features))
        return combined_features.astype(np.float32)


In [7]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

class NeutrinoModel(pl.LightningModule):
    def __init__(self, input_dims, num_layers=2, hidden_units=None, learning_rate=0.001):
        super().__init__()
        self.save_hyperparameters()

        layers = []
        layers.append(nn.BatchNorm1d(input_dims))

        if hidden_units is None:
            hidden_units = [input_dims*2] * (num_layers - 1)

        for i in range(num_layers):
            in_dim = input_dims if i == 0 else hidden_units
            out_dim = 2 if i == num_layers - 1 else hidden_units
            layers.append(nn.Linear(in_dim, out_dim))

            if i < num_layers - 1:
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Sigmoid())

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.l1_loss(y_hat, y, reduction='mean')
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_norm = batch
        y_hat_norm = self(x)
        loss = F.l1_loss(y_hat_norm, y_norm, reduction='mean')

        # Denormalize target values and predictions
        y = denormalize_targets(y_norm.cpu().numpy())
        y_hat = denormalize_targets(y_hat_norm.detach().cpu().numpy())

        # Calculate angular distance score
        az_true, zen_true = y[:, 0], y[:, 1]
        az_pred, zen_pred = y_hat[:, 0], y_hat[:, 1]
        ang_dist = angular_dist_score(az_true, zen_true, az_pred, zen_pred)

        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("angular_dist_score", ang_dist, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)


In [9]:
def train_one_epoch_on_batch(model, metadata, sensor_geometry, batch_file, val_split=0.2):
    batch_id = int(batch_file.split('_')[-1].split('.')[0])
    batch_metadata = metadata[metadata['batch_id'] == batch_id]
    
    train_metadata, val_metadata = train_test_split(batch_metadata, test_size=val_split, random_state=42)
    
    train_dataset = NeutrinoDataset(train_metadata, sensor_geometry, batch_file, mode="train")
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    
    val_dataset = NeutrinoDataset(val_metadata, sensor_geometry, batch_file, mode="train")
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

    trainer = pl.Trainer(max_epochs=1)
    trainer.fit(model, train_dataloader, val_dataloader)
    
    del train_dataset, train_dataloader, val_dataset, val_dataloader


In [10]:
data_dir = '/kaggle/input/icecube-neutrinos-in-deep-ice/'
metadata_file = f'{data_dir}/train_meta.parquet'
batch_files = [f'{data_dir}train/batch_{i}.parquet' for i in range(1, 2)] # You can set up to 660 files

sensor_geometry = pd.read_csv(os.path.join(data_dir, 'sensor_geometry.csv'))
metadata = pd.read_parquet(metadata_file)

In [11]:
config = {
    "input_dims": 170,
    "num_layers": 4,
    "hidden_units": 512,
    "learning_rate": 0.001,
}

model = NeutrinoModel(
    input_dims=config['input_dims'],
    num_layers=config['num_layers'],
    hidden_units=config['hidden_units'],
    learning_rate=config['learning_rate']
)

In [12]:
for epoch in range(1): # You can set any number of epochs but first use all of 660 files
    for batch_file in batch_files:
        train_one_epoch_on_batch(model, metadata, sensor_geometry, batch_file)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [13]:
def predict(model, metadata, sensor_geometry, batch_files, output_file="submission.csv"):
    model.eval()
    
    with open(output_file, 'w') as f:
        f.write('event_id,azimuth,zenith\n')

    for batch_file in batch_files:
        batch_id = int(batch_file.split('_')[-1].split('.')[0])
        batch_metadata = metadata[metadata['batch_id'] == batch_id]
        test_dataset = NeutrinoDataset(batch_metadata, sensor_geometry, batch_file, mode="test")
        test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=4)
        
        for event_ids, features in test_dataloader:
            with torch.no_grad():
                pred_norm = model(features)
                pred = denormalize_targets(pred_norm.detach().numpy())
                
                with open(output_file, 'a') as f:
                    for event_id, prediction in zip(event_ids, pred):
                        f.write(f'{event_id},{prediction[0]},{prediction[1]}\n')

In [14]:
test_metadata_file = f'{data_dir}/test_meta.parquet'
test_metadata = pd.read_parquet(test_metadata_file)


test_dir = '/kaggle/input/icecube-neutrinos-in-deep-ice/test/'
test_batch_files = [f'{test_dir}/{file}' for file in os.listdir(test_dir)]

predict(model, test_metadata, sensor_geometry, test_batch_files, "submission.csv")

In [15]:
pd.read_csv('submission.csv')

,event_id,azimuth,zenith
0,2092,3.484541,2.102088
1,7344,3.258514,1.497922
2,9482,2.114624,1.478094
